In [9]:
import numpy as np
data1 = np.array([100,32,400,500,150,40])
data2 = np.array([32,29,65,100,123]) 
th = 50

def filter_irisan(data1,data2,th):
    filter_data1 = data1[data1>th]
    filter_data2 = data2[data2>th]
    output = []
    for i in filter_data1:
        if i in filter_data2:
            output.append(i)
    return output

filter_irisan(data1,data2,th)

[100]

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset,DataLoader
import numpy as np
from torch.utils.data import Dataset
from PIL import Image
from torchvision.models import efficientnet_b3
import torch.nn.functional as F
import matplotlib.pyplot as plt
import pandas as pd
import os
import torchvision.utils as vutils
from models import Discriminator, Generator, initialize_weights
from utils import gradient_penalty

In [21]:
date_today = "24 May 23"
jenis_dataset = "dog_anime_celeb"
nama_dataset = ["anime", "celeb", "dog"]

In [22]:
path_cgan2 = "C:/Users/62812/Documents/Kuliah/Semester 2/Deep Learning/Tugas 4/Controlable GAN2/Dataset 2"

In [23]:
device = "cuda" if torch.cuda.is_available() else "cpu"
LEARNING_RATE = 1e-4
BATCH_SIZE = 64
IMG_SIZE = 64 #coba 64 aslinya 224
CHANNELS_IMG = 3
Z_DIM = 100
NUM_EPOCHS = 1000
FEATURES_CRITIC = 16
FEATURES_GEN = 16
CRITIC_ITERATIONS = 5
LAMBDA_GP = 10
NUM_CLASSES = len(nama_dataset)
EMBED_SIZE = len(nama_dataset)

In [25]:
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE,IMG_SIZE)), #coba ganti  64, harusnya 224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

def create_dataset(path,nama_dataset,transform):
    files = os.listdir(path)
    images = []
    labels = []
    for file in files:
        label = nama_dataset.index(file)
        file_images = os.listdir(f"{path}/{file}")
        for img in file_images:
            img_i = Image.open(f"{path}/{file}/{img}")
            images.append(transform(img_i))
            labels.append(label)
    stacked_tensor = torch.stack([tensor_img for tensor_img in images], dim=0)
    return stacked_tensor,torch.tensor(labels)

In [26]:
X , label = create_dataset(path_cgan2, nama_dataset, transform)

In [27]:
jumlah_per_jenis_gambar = {}
for n,jenis_gambar in enumerate(nama_dataset):
    jumlah_per_jenis_gambar[jenis_gambar] = len(label[label==n])
print(jumlah_per_jenis_gambar)    

{'anime': 1000, 'celeb': 1000, 'dog': 821}


In [28]:
# Create a TensorDataset object
dataset = TensorDataset(X, label)

# Create a DataLoader object with batch size and shuffle=True
dataloader = DataLoader(dataset, batch_size = BATCH_SIZE, shuffle=True)

In [29]:
# initialize gen and disc, note: discriminator should be called critic,
# according to WGAN paper (since it no longer outputs between [0, 1])
gen = Generator(Z_DIM, CHANNELS_IMG, FEATURES_GEN, NUM_CLASSES,IMG_SIZE, EMBED_SIZE).to(device)
critic = Discriminator(CHANNELS_IMG, FEATURES_CRITIC,NUM_CLASSES,IMG_SIZE).to(device)
initialize_weights(gen)
initialize_weights(critic)

# initializate optimizer
opt_gen = optim.Adam(gen.parameters(), lr=LEARNING_RATE, betas=(0.0, 0.9))
opt_critic = optim.Adam(critic.parameters(), lr=LEARNING_RATE, betas=(0.0, 0.9))

In [10]:
import time
path_eksperimen = "C:/Users/62812/Documents/Kuliah/Semester 2/Deep Learning/Tugas 4/Controlable GAN2"

In [11]:
# import time
# path_eksperimen = "C:/Users/62812/Documents/Kuliah/Semester 2/Deep Learning/Tugas 4/Controlable GAN2"
# t_start = time.time()
# # for tensorboard plotting
# fixed_noise = torch.randn(BATCH_SIZE, Z_DIM, 1, 1).to(device)

# gen.train()
# critic.train()

# for epoch in range(100,NUM_EPOCHS):
#     for batch_idx, (real, labels) in enumerate(dataloader):
#         real = real.to(device)
#         cur_batch_size = real.shape[0]
#         labels = labels.to(device)
#         # Train Critic: max E[critic(real)] - E[critic(fake)]
#         # equivalent to minimizing the negative of that
#         for _ in range(CRITIC_ITERATIONS):
#             noise = torch.randn(cur_batch_size, Z_DIM, 1, 1).to(device)
#             fake = gen(noise,labels)
#             critic_real = critic(real,labels).reshape(-1)
#             critic_fake = critic(fake,labels).reshape(-1)
#             gp = gradient_penalty(critic,labels, real, fake, device)
#             loss_critic = (
#                 -(torch.mean(critic_real) - torch.mean(critic_fake)) + LAMBDA_GP * gp
#             )
#             critic.zero_grad()
#             loss_critic.backward(retain_graph=True)
#             opt_critic.step()

#         # Train Generator: max E[critic(gen_fake)] <-> min -E[critic(gen_fake)]
#         gen_fake = critic(fake,labels).reshape(-1)
#         loss_gen = -torch.mean(gen_fake)
#         gen.zero_grad()
#         loss_gen.backward()
#         opt_gen.step()
#         if batch_idx == 42 and epoch % 10 == 0: 
#             with torch.no_grad():
#                 print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] with  Loss D: {loss_critic:.4f}, loss G: {loss_gen:.4f}")
#                 vutils.save_image(real,f"{path_eksperimen}/Sample Image/{date_today} - real samples epoch {epoch} batch {batch_idx}.png", normalize = True)
#                 fake = gen(noise, labels)
#                 vutils.save_image(fake, f"{path_eksperimen}/Fake Image/{date_today} - fake samples epoch {epoch} batch {batch_idx}.png",normalize = True)
                
# t_end = time.time()
# lama = t_end - t_start
# print(f"Waktu running {NUM_EPOCHS} menggunakan {device} yaitu {lama} detik")

In [23]:
#save model
discriminator_path = f"{path_eksperimen}/Discriminator Model/{date_today}_disc_{jenis_dataset}_embed_size_{EMBED_SIZE}_epoch_{NUM_EPOCHS}.pth"
generator_path = f"{path_eksperimen}/Generator Model/{date_today}_gen_{jenis_dataset}_embed_size_{EMBED_SIZE}_epoch_{NUM_EPOCHS}.pth"
torch.save(critic.state_dict(), discriminator_path)
torch.save(gen.state_dict(), generator_path)

In [17]:
def gen_gambar(model,class_predict,jumlah_gambar, nama_dataset, path, embed_size,NUM_EPOCHS):
    with torch.no_grad():
        noise = torch.randn(jumlah_gambar, Z_DIM, 1, 1).to(device)
        idx_encode = nama_dataset.index(class_predict)
        encode = [idx_encode for _ in range(jumlah_gambar)]
        labels = torch.tensor(encode).to(device)
        pic_predict = model(noise,labels)
        vutils.save_image(pic_predict, f"{path} {date_today} - Hasil predict gambar {class_predict} embed size {embed_size} epoch {NUM_EPOCHS}.png", normalize = True)

In [18]:
path_model_generator = "C:/Users/62812/Documents/Kuliah/Semester 2/Deep Learning/Tugas 4/Controlable GAN2/Generator Model/19 May 23_gen_dog_anime_celeb_embed_size_3_epoch_1000.pth"
gen.load_state_dict(torch.load(path_model_generator))
gen.eval()

Generator(
  (net): Sequential(
    (0): Sequential(
      (0): ConvTranspose2d(103, 256, kernel_size=(4, 4), stride=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (1): Sequential(
      (0): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (2): Sequential(
      (0): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (3): Sequential(
      (0): ConvTranspose2d(64, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (4): ConvTranspose2d(32, 3, kernel_si

In [19]:
path_test = "C:/Users/62812/Documents/Kuliah/Semester 2/Deep Learning/Tugas 4/Controlable GAN2/Test Image/"
gen_gambar(gen,"Anime", nama_dataset ,64, path_test, EMBED_SIZE, NUM_EPOCHS)

TypeError: randn() received an invalid combination of arguments - got (list, int, int, int), but expected one of:
 * (tuple of ints size, *, torch.Generator generator, tuple of names names, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, torch.Generator generator, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, tuple of names names, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [ ]:
for class_predict in nama_dataset:
    path_test = "C:/Users/62812/Documents/Kuliah/Semester 2/Deep Learning/Tugas 4/Controlable GAN2/Test Image/"
    gen_gambar(gen,class_predict, nama_dataset, path_test, EMBED_SIZE, NUM_EPOCHS)
    

In [1]:
import numpy as np
np.exp(0.5)

1.6487212707001282

In [2]:
np.exp(0)

1.0